In [1]:
!python ../iuxray/extract_findings_and_impressions.py

Loading file /mnt/workspace/iu-x-ray/dataset/reports/reports.json ...
Extracing and merging findings and impressions ...
Saving to /home/pamessina/medvqa-workspace/cache/iuxray/findings+impression.txt ...
done!


In [2]:
!python ../mimiccxr/extract_findings_and_impressions.py

Loading MIMIC-CXR's report file paths ...
227835it [02:43, 1394.99it/s] 
Saving findings + impressions to /home/pamessina/medvqa-workspace/cache/mimiccxr/findings+impression.txt ...
100%|██████████████████████████████████| 227835/227835 [09:44<00:00, 390.11it/s]
Num failed paths = 419
Done!


In [1]:
from importlib import reload

In [2]:
from medvqa.utils.files import read_lines_from_txt
from medvqa.utils.common import CACHE_DIR
from medvqa.datasets.qa_pairs_extractor import QuestionAnswerExtractor
import medvqa
import re
import os
import random
import math
from tqdm import tqdm

In [586]:
reload(medvqa.datasets.qa_pairs_extractor)

<module 'medvqa.datasets.qa_pairs_extractor' from '/home/pamessina/medvqa/medvqa/datasets/qa_pairs_extractor.py'>

In [3]:
iuxray_raw_reports = read_lines_from_txt(os.path.join(CACHE_DIR, 'iuxray', 'findings+impression.txt'))

In [4]:
mimiccxr_raw_reports = read_lines_from_txt(os.path.join(CACHE_DIR, 'mimiccxr', 'findings+impression.txt'))

In [5]:
len(iuxray_raw_reports), len(mimiccxr_raw_reports)

(3927, 227416)

In [6]:
def print_qa_pairs(qa_pairs):
    for k, v in qa_pairs.items():
        print('------------')
        print('Q:', k)
        for s in v:
            print('A:',s)

In [7]:
qa_extractor = QuestionAnswerExtractor()

In [8]:
connectors = ['when', 'and', 'but', 'however', 'although', 'should', 'since', 'unless', 'nevertheless']
exp = '('+'|'.join(f'\\b{x}\\b' for x in connectors)+'|[,;.:])'
full_exp = f'{exp}+(\\s+{exp}+)*'
sep_regex = re.compile(full_exp, re.IGNORECASE)
sep_regex

re.compile(r'(\bwhen\b|\band\b|\bbut\b|\bhowever\b|\balthough\b|\bshould\b|\bsince\b|\bunless\b|\bnevertheless\b|[,;.:])+(\s+(\bwhen\b|\band\b|\bbut\b|\bhowever\b|\balthough\b|\bshould\b|\bsince\b|\bunless\b|\bnevertheless\b|[,;.:])+)*',
re.IGNORECASE|re.UNICODE)

In [9]:
def clean_sentence(self, sentence):
    last_sep = None
    i = 0
    n = len(sentence)
    clean_sentence = ''
    prepend_to_next_chunk = None
    for match in sep_regex.finditer(sentence):
        span = match.span()
        if i < span[0]:
            chunk = sentence[i:span[0]].strip()
            if prepend_to_next_chunk:
                if len(clean_sentence) > 0:
                    chunk = prepend_to_next_chunk + chunk
                prepend_to_next_chunk = None
            if self._is_relevant(chunk):
                if len(clean_sentence) > 0:
                    if last_sep:
                        clean_sentence += ' ' + last_sep
                    clean_sentence += ' '                    
                clean_sentence += chunk
            print('chunk =', chunk)
        i = span[1]
        last_sep = sentence[span[0]:span[1]]
        print('last_sep =', last_sep)
        if last_sep not in ('and', 'but', ':'):
            if last_sep == ', however,':
                last_sep = None
            else:
                done = False
                for x in ('should', 'since', 'unless', 'when'):
                    if x in last_sep:
                        prepend_to_next_chunk = last_sep + ' '
                        last_sep = None
                        done = True
                        break
                if not done:
                    last_sep = ','
    if i < n:
        chunk = sentence[i:].strip()
        if prepend_to_next_chunk:
            chunk = prepend_to_next_chunk + chunk
        if self._is_relevant(chunk):
            if len(clean_sentence) > 0:
                if last_sep:
                    clean_sentence += ' ' + last_sep
                clean_sentence += ' '
            clean_sentence += chunk
        print('chunk =', chunk)
    return clean_sentence

In [10]:
_s = 'AP chest compared to through : Pulmonary vascular congestion and mediastinal venous engorgement have progressed since .'
for x in sep_regex.finditer(_s):
    print(str(x))
print("$$$$"+clean_sentence(qa_extractor, _s)+"$$$$")
print("$$$$"+qa_extractor.clean_sentence(_s)+"$$$$")

<_sre.SRE_Match object; span=(29, 30), match=':'>
<_sre.SRE_Match object; span=(61, 64), match='and'>
<_sre.SRE_Match object; span=(112, 119), match='since .'>
chunk = AP chest compared to through
last_sep = :
chunk = Pulmonary vascular congestion
last_sep = and
chunk = mediastinal venous engorgement have progressed
last_sep = since .
$$$$Pulmonary vascular congestion and mediastinal venous engorgement have progressed$$$$
$$$$Pulmonary vascular congestion and mediastinal venous engorgement have progressed$$$$


In [11]:
tmp = qa_extractor.generate_qa_pairs('Lung volumes continues to be low.  The heart continues to be moderately enlarged with unchanged mild to moderate edema.  Moderate bilateral, right greater than left, pleural effusions are unchanged.  ET tube is in appropriate position, and the gastric tube ends in the stomach.  Left central venous line is in appropriate position.  Right PICC line ends at the lower SVC. Unchanged low lung volumes, mild pulmonary edema and bilateral moderate pleural effusions. ', debug=True)
print_qa_pairs(tmp)

------------
Q: lungs?
A: Lung volumes continues to be low
A: Unchanged low lung volumes , mild pulmonary edema and bilateral moderate pleural effusions
------------
Q: heart?
A: The heart continues to be moderately enlarged with unchanged mild to moderate edema
------------
Q: edema?
A: The heart continues to be moderately enlarged with unchanged mild to moderate edema
A: Unchanged low lung volumes , mild pulmonary edema and bilateral moderate pleural effusions
------------
Q: pleural space?
A: Moderate bilateral , right greater than left , pleural effusions are unchanged
A: Unchanged low lung volumes , mild pulmonary edema and bilateral moderate pleural effusions
------------
Q: tubes and lines?
A: ET tube is in appropriate position , the gastric tube ends in the stomach
A: Left central venous line is in appropriate position
A: Right PICC line ends at the lower SVC
------------
Q: support devices and foreign bodies?
A: ET tube is in appropriate position , the gastric tube ends in the

In [13]:
tmp = qa_extractor.generate_qa_pairs('Lung volumes continues to be low.  The heart continues to be moderately enlarged with unchanged mild to moderate edema.  Moderate bilateral, right greater than left, pleural effusions are unchanged.  ET tube is in appropriate position, and the gastric tube ends in the stomach.  Left central venous line is in appropriate position.  Right PICC line ends at the lower SVC. Unchanged low lung volumes, mild pulmonary edema and bilateral moderate pleural effusions. ', debug=True)
print_qa_pairs(tmp)

------------
Q: lungs?
A: Lung volumes continues to be low
A: Unchanged low lung volumes , mild pulmonary edema and bilateral moderate pleural effusions
------------
Q: heart?
A: The heart continues to be moderately enlarged with unchanged mild to moderate edema
------------
Q: edema?
A: The heart continues to be moderately enlarged with unchanged mild to moderate edema
A: Unchanged low lung volumes , mild pulmonary edema and bilateral moderate pleural effusions
------------
Q: pleural space?
A: Moderate bilateral , right greater than left , pleural effusions are unchanged
A: Unchanged low lung volumes , mild pulmonary edema and bilateral moderate pleural effusions
------------
Q: tubes and lines?
A: ET tube is in appropriate position , the gastric tube ends in the stomach
A: Left central venous line is in appropriate position
A: Right PICC line ends at the lower SVC
------------
Q: support devices and foreign bodies?
A: ET tube is in appropriate position , the gastric tube ends in the

In [14]:
report = random.choice(mimiccxr_raw_reports)
print(report)
print('=============================')
tmp = qa_extractor.generate_qa_pairs(report, debug=True)
print_qa_pairs(tmp)

PA and lateral views of the chest demonstrate the lungs are well expanded. Apparent bulging of the posterior left mediastinum adjacent to the descending aorta, has been attributed to varices on prior studies. There is no evidence of focal consolidation, effusion, or pneumothorax. Mild interstitial prominence is present, most notably in the right lower lung, with no evidence of overt pulmonary edema. Mild interstitial prominence with no evidence of overt pulmonary edema. No pneumonia is present.
------------
Q: lungs?
A: lateral views of the chest demonstrate the lungs are well expanded
A: Mild interstitial prominence is present , most notably in the right lower lung , with no evidence of overt pulmonary edema
A: Mild interstitial prominence with no evidence of overt pulmonary edema
------------
Q: thoracic aorta?
A: Apparent bulging of the posterior left mediastinum adjacent to the descending aorta
------------
Q: mediastinum?
A: Apparent bulging of the posterior left mediastinum adjac

In [17]:
tmp = qa_extractor.generate_qa_pairs_compact_version(report)
tmp

{'sentences': ['lateral views of the chest demonstrate the lungs are well expanded',
  'Apparent bulging of the posterior left mediastinum adjacent to the descending aorta',
  'There is no evidence of focal consolidation , effusion , or pneumothorax',
  'Mild interstitial prominence is present , most notably in the right lower lung , with no evidence of overt pulmonary edema',
  'Mild interstitial prominence with no evidence of overt pulmonary edema',
  'No pneumonia is present'],
 'invalid': [],
 'unmatched': [],
 'matched': [0, 1, 2, 3, 4, 5],
 'qa': {'59': [0, 3, 4],
  '90': [1],
  '63': [1],
  '68': [2],
  '73': [2],
  '32': [2],
  '36': [3, 4],
  '74': [3, 4],
  '52': [3, 4],
  '71': [5]},
 'question_ids': [59, 90, 63, 68, 73, 32, 36, 74, 52, 71]}

In [13]:
_s = 'the enlarging pleural effusion has displaced the cardiomediastinal contours towards the left , limiting assessment of the heart size'
print("$$$$"+qa_extractor.clean_sentence(_s)+"$$$$")

$$$$the enlarging pleural effusion has displaced the cardiomediastinal contours towards the left$$$$


In [14]:
len(qa_extractor.questions), qa_extractor.questions

(97,
 ['ARDS?',
  'COPD?',
  'abscess and cavitation?',
  'adenopathy?',
  'air collections?',
  'air space disease?',
  'air-fluid level?',
  'airways?',
  'apical zone?',
  'ascites?',
  'aspiration?',
  'atelectasis?',
  'azygos lobe?',
  'azygos vein?',
  'bleeding?',
  'blurring?',
  'bones?',
  'bowel obstruction and loops?',
  'bowel?',
  'bronchiectasis?',
  'bronchogram?',
  'bronchovascular crowding?',
  'bronchus?',
  'bullae and blebs?',
  'calcification?',
  'cancer or tumor?',
  'cardiac silhouette?',
  'cardiomegaly?',
  'carina?',
  'cholelithiasis?',
  'colon?',
  'congestive heart failure CHF?',
  'consolidation?',
  'contrast?',
  'densities?',
  'diaphragm?',
  'edema?',
  'emphysema?',
  'esophagus?',
  'fibrosis?',
  'fissures?',
  'fluid overload?',
  'fluid?',
  'fractures?',
  'free air?',
  'gas distension?',
  'granumola?',
  'haze?',
  'heart?',
  'hernias?',
  'infection?',
  'infiltrate?',
  'interstitial lung disease?',
  'kerley lines?',
  'lesions?',
  

In [15]:
tmp = qa_extractor.search_by_unmatched(0, mimiccxr_raw_reports, 100)
x = tmp[1]
tmp

(2, 170895)

In [16]:
qa_extractor.generate_qa_pairs_compact_version(mimiccxr_raw_reports[x])

{'sentences': ['Left hip replacement',
  'Portable AP radiograph of the chest was reviewed in comparison to .',
  'The ET tube tip is 6 , 5 cm above the carina',
  'Heart size and mediastinum are stable',
  'Bibasilar areas of atelectasis are noted , might be reflecting aspiration',
  'Attention to these areas on the subsequent studies is recommended'],
 'invalid': [1],
 'unmatched': [0, 5],
 'matched': [2, 3, 4],
 'qa': {'95': [2],
  '87': [2],
  '28': [2],
  '48': [3],
  '63': [3],
  '59': [4],
  '11': [4],
  '10': [4]}}

In [17]:
# _report = ""
# print(_report)
# _qa_pairs = qa_generator.generate_question_answer_pairs(_report)
print(mimiccxr_raw_reports[x])
print("======================")
_qa_pairs = qa_extractor.generate_qa_pairs(mimiccxr_raw_reports[x], debug=True)
# _qa_pairs = qa_generator.remove_almost_duplicate_sentences(_qa_pairs)
print_qa_pairs(_qa_pairs)

Left hip replacement. Portable AP radiograph of the chest was reviewed in comparison to . The ET tube tip is 6.5 cm above the carina. Heart size and mediastinum are stable. Bibasilar areas of atelectasis are noted, new since the prior study, but might be reflecting aspiration. Attention to these areas on the subsequent studies is recommended.
---- invalid: Portable AP radiograph of the chest was reviewed in comparison to .
** not captured: Left hip replacement
** not captured: Attention to these areas on the subsequent studies is recommended
------------
Q: tubes and lines?
A: The ET tube tip is 6 , 5 cm above the carina
------------
Q: support devices and foreign bodies?
A: The ET tube tip is 6 , 5 cm above the carina
------------
Q: carina?
A: The ET tube tip is 6 , 5 cm above the carina
------------
Q: heart?
A: Heart size and mediastinum are stable
------------
Q: mediastinum?
A: Heart size and mediastinum are stable
------------
Q: lungs?
A: Bibasilar areas of atelectasis are note

In [19]:
def get_informative_unmatched_sentences(corpus, qa_extractor, n_samples = 2000):
    
    indices = range(len(corpus))
    if len(corpus) > n_samples:
        indices = random.sample(indices, n_samples)
        
    unmatched_sentences = set()
    word2freq = dict()
    
    for i in tqdm(indices):
        for s in qa_extractor.get_unmatched_sentences(corpus[i]):
            unmatched_sentences.add(s)
            for w in re.split(r",?\s+", s):
                word2freq[w] = word2freq.get(w, 0) + 1
    
    max_freq = max(word2freq.values())    
    unmatched_sentences = list(unmatched_sentences)
    
    def score(s):
        score_max = 0
        score_sum = 0
        cnt = 0
        for w in re.split(r",?\s+", s):
            if w not in qa_extractor.stopwords:
                tmp = max_freq - word2freq.get(w, 0)
                score_max = max(tmp, score_max)
                score_sum += tmp
                cnt += 1
        return score_max * 0.4 + (score_sum / math.sqrt(max(cnt,1))) * 0.6
    
    unmatched_sentences.sort(key=score, reverse=True)
    return unmatched_sentences

In [20]:
unmatched_sentences = get_informative_unmatched_sentences(mimiccxr_raw_reports, qa_extractor, n_samples=1000)

100%|██████████| 1000/1000 [00:06<00:00, 143.00it/s]


In [21]:
len(unmatched_sentences)

209

In [22]:
qa_extractor.reload()

In [23]:
print_qa_pairs(qa_extractor.generate_qa_pairs('. '.join(unmatched_sentences[0:40]), debug=True))

** not captured: The distance between the superior margin of the lung and the outer chest wall is about 15 mm , which is slightly increased
** not captured: oblique and lordotic chest x-ray views to determine whether this lies within the lung or the bone
** not captured: the left hemithorax has increased in transparency , likely reflecting improved ventilation
** not captured: lateral chest radiographs are limited by patient's body habitus and inability to raise arms
** not captured: Appears to cross the midline , there is concern for potential arterial location
** not captured: Tip probably does not enter the transverse portion of the left brachiocephalic vein
** not captured: there is a minimal improvement in transparency , likely reflecting improved ventilation
** not captured: lordotic views to determine whether this lies within the lung or the bone
** not captured: Recommend withdrawal by approximately 2 cm for optimal positioning
** not captured: This could be advanced several cm

In [18]:
!python ../../scripts/iuxray/generate_qa_pairs_from_reports.py

Loading original reports from /mnt/workspace/iu-x-ray/dataset/reports/reports.json
Processing each report ...
100%|██████████████████████████████████████| 3955/3955 [00:36<00:00, 107.62it/s]
Saving qa adapted reports to /home/pamessina/medvqa-workspace/cache/iuxray/qa_adapted_reports__20220428_181414.json
Done!


In [ ]:
!python ../../scripts/mimiccxr/generate_qa_pairs_from_reports.py

Processing MIMIC-CXR's original reports ...
6533it [01:56, 78.90it/s]

## Generate summary file

In [3]:
from medvqa.utils.files import load_json_file

In [14]:
iuxray_qa_adapted_reports = load_json_file('/home/pamessina/medvqa-workspace/cache/iuxray/qa_adapted_reports__20220416_072043.json')

In [15]:
mimiccxr_qa_adapted_reports = load_json_file('/home/pamessina/medvqa-workspace/cache/mimiccxr/qa_adapted_reports__20220416_083114.json')

In [6]:
len(iuxray_qa_adapted_reports['questions']), len(iuxray_qa_adapted_reports['reports'])

(97, 3927)

In [16]:
len(mimiccxr_qa_adapted_reports['questions']), len(mimiccxr_qa_adapted_reports['reports'])

(97, 227835)

In [17]:
iuxray_qa_adapted_reports['reports'][0]

{'sentences': ['The cardiac silhouette and mediastinum size are within normal limits',
  'There is no pulmonary edema',
  'There is no focal consolidation',
  'There are no XXXX of a pleural effusion.',
  'There is no evidence of pneumothorax',
  'Normal chest x-XXXX.'],
 'invalid': [3, 5],
 'unmatched': [],
 'matched': [0, 1, 2, 4],
 'qa': {'26': [0], '63': [0], '36': [1], '32': [2], '73': [4]},
 'filename': '1.xml'}

In [18]:
mimiccxr_qa_adapted_reports['reports'][0]

{'sentences': ['Heart size is normal',
  'There are mild aortic knob calcifications',
  'Cardiomediastinal silhouette and hilar contours are otherwise unremarkable',
  'Lungs are clear',
  'Pleural surfaces are clear without effusion or pneumothorax',
  'No acute cardiopulmonary abnormality'],
 'invalid': [],
 'unmatched': [],
 'matched': [0, 1, 2, 3, 4, 5],
 'qa': {'48': [0, 5],
  '90': [1],
  '24': [1],
  '75': [2],
  '26': [2],
  '63': [2],
  '59': [3, 5],
  '68': [4],
  '73': [4]},
 'filepath': '/mnt/data/mimic-cxr/files/p10/p10703179/s58829627.txt'}

In [19]:
def generate_summary_file(dataset, k, kk, output_path):
    
    q2a = [[] for _ in range(len(dataset['questions']))]
    n_invalid = 0
    unmatched_sentences = []
    for ri, report in enumerate(dataset['reports']):
        s = report['sentences']
        for q_idx, a_idxs in report['qa'].items():
            q_idx = int(q_idx)
            a_len = sum(len(s[i]) for i in a_idxs)
            q2a[q_idx].append((a_len, ri))
        n_invalid += len(report['invalid'])
        unmatched_sentences.extend(s[i] for i in report['unmatched'])
            
    sorted_questions = sorted([(len(a), i) for i,a in enumerate(q2a)], reverse=True)    
    
    with open(output_path, 'w') as file:
        file.write('Preguntas ordenadas por frecuencia (decreciente)\n\n')
        for f, q in sorted_questions:
            file.write('%d, %s\n' % (f, dataset['questions'][q]))
        file.write('\n')
        file.write('Ejemplos de respuestas por pregunta:')
        for i,(_,q) in enumerate(sorted_questions):
            answers = q2a[q]
            answers.sort()            
            tmp = []
            for _, a in answers:
                report = dataset['reports'][a]
                sentences = report['sentences']
                a_idxs = report['qa'][str(q)]
                answer = '. '.join(sentences[i] for i in a_idxs)
                tmp.append(answer)
            answers = tmp            
            n = len(answers)
            print("n=%d, %s" % (n, dataset['questions'][q]))
            if n > k:
                sample = []
                for j in range(k):
                    sample.append(answers[random.randint(int(n*j/k), int(n*(j+1)/k)-1)])
            else:
                sample = answers
            file.write('\n\n%d) %s:\n\n' % (i+1, dataset['questions'][q]))
            for a in sample:
                file.write('    A: %s\n' % a)
        file.write('\n\nTotal oraciones inválidas: %d\n' % n_invalid)
        file.write('Total oraciones sin match: %d\n' % len(unmatched_sentences))
        file.write('\n\nEjemplos de oraciones válidas que no hicieron match con ninguna pregunta\n\n')
        unmatched_sentences = list(set(unmatched_sentences))
        unmatched_sentences.sort(key=lambda s:len(s))
        nn = len(unmatched_sentences)
        for j in range(kk):
            file.write('    %s\n' % unmatched_sentences[random.randint(int(nn*j/kk), int(nn*(j+1)/kk)-1)])

In [20]:
generate_summary_file(iuxray_qa_adapted_reports, 10, 120, '../iuxray/iuxray_qa_dataset_summary_file.txt')

n=3514, lungs?
n=3343, heart?
n=3053, pleural space?
n=2686, pneumothorax?
n=2345, mediastinum?
n=1934, bones?
n=1224, consolidation?
n=1180, cardiac silhouette?
n=796, pulmonary vascularity?
n=714, opacities?
n=546, air space disease?
n=540, calcification?
n=435, granumola?
n=414, infiltrate?
n=408, support devices and foreign bodies?
n=389, thoracic aorta?
n=360, atelectasis?
n=348, edema?
n=328, nodules?
n=303, cardiomegaly?
n=257, pneumonia?
n=239, diaphragm?
n=223, tortuosity?
n=220, surgery?
n=213, interstitial lung disease?
n=207, scarring?
n=207, pulmonary hila?
n=204, tubes and lines?
n=192, fractures?
n=190, ribs?
n=182, masses?
n=144, prominence?
n=138, soft tissue?
n=135, densities?
n=122, emphysema?
n=109, trachea?
n=107, apical zone?
n=98, adenopathy?
n=97, bronchus?
n=88, markings?
n=82, bronchovascular crowding?
n=74, fluid?
n=65, infection?
n=64, free air?
n=61, air collections?
n=57, lesions?
n=53, hernias?
n=48, lung parenchyma?
n=46, vena cava?
n=43, COPD?
n=42, fib

In [21]:
generate_summary_file(mimiccxr_qa_adapted_reports, 10, 120, '../mimiccxr/mimiccxr_qa_dataset_summary_file.txt')

n=191641, lungs?
n=184260, pleural space?
n=144619, pneumothorax?
n=129244, heart?
n=117811, mediastinum?
n=107593, support devices and foreign bodies?
n=97824, cardiac silhouette?
n=86492, tubes and lines?
n=72867, bones?
n=69556, opacities?
n=69505, consolidation?
n=69001, atelectasis?
n=65524, edema?
n=54153, pulmonary hila?
n=53740, pulmonary vascularity?
n=52914, pneumonia?
n=31810, cardiomegaly?
n=28096, surgery?
n=25534, diaphragm?
n=22222, thoracic aorta?
n=20327, vena cava?
n=17058, stomach?
n=16204, trachea?
n=15391, interstitial lung disease?
n=14336, calcification?
n=13946, apical zone?
n=13616, fractures?
n=13425, infection?
n=11897, carina?
n=11465, lung parenchyma?
n=11200, ribs?
n=11120, tortuosity?
n=10721, prominence?
n=9560, free air?
n=9184, fluid?
n=8704, bronchus?
n=8331, aspiration?
n=7628, scarring?
n=6981, densities?
n=6901, nodules?
n=5485, emphysema?
n=5371, markings?
n=5371, congestive heart failure CHF?
n=4781, infiltrate?
n=4594, esophagus?
n=4186, air col